In [1]:
import numpy as np
import pandas as pd
import descartes
# import geopandas as gpd
# from shapely.geometry import Point, Polygon

In [2]:
df = pd.read_csv('ghcnd_hcn/pair_final.csv')
df.shape

(524065, 15)

In [3]:
distance_arr = []
test_df = df.query("STATION == 'USC00057936'")
test_location = np.array([test_df[element].values[0] for element in ['LON', 'LAT', 'ELEV']])

k = 3

for station_id, station_name in enumerate(df['STATION'].unique()):
    nearby_df = df.query("STATION == '{}'".format(station_name))
    nearby_location = np.array([nearby_df[element].values[0] for element in ['LON', 'LAT', 'ELEV']])
    nearby_distance = np.linalg.norm(test_location - nearby_location)
    
    distance_arr.append(nearby_distance)

k_nearest_stations = df['STATION'].unique()[np.argpartition(distance_arr, k)[:k]]

In [4]:
nearby_stations_df = df.query("STATION in @k_nearest_stations")

In [5]:
df_train = []
df_test = []
for station in k_nearest_stations:
    print(station)
    if station != "USC00057936":
        df_temp = df.query("STATION == '{}'".format(station))
        df_temp = df_temp.set_index(['DATE'])
        df_train.append(df_temp)
    else:
        df_test = df.query("STATION == '{}'".format(station))
        df_test = df_test.set_index(['DATE'])

USC00057936
USC00051528
USC00023596


In [6]:
Y = pd.DataFrame(df_test['PRCP2'])
Y.columns = ['Precipitation']

In [7]:
df_train[1].columns = ['2STATION', '2LAT', '2LON', '2ELEV', '2PRCP1', '2TMAX1', '2TMIN1', '2SNOW1',
       '2SNOWD1', '2PRCP2', 'TMAX2', 'TMIN2', 'SNOW2', 'SNOWD2']
df_train.append(Y)
nearby_final = pd.concat(df_train, axis=1,join='inner', sort=False)

In [14]:
nearby_final = nearby_final.reset_index()
nearby_final = nearby_final.select_dtypes(exclude=['object'])